In [17]:
import subprocess
import os
import TSED
import torch
from scipy.spatial.distance import cosine
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4")
import os
from openai import OpenAI
import json

from transformers import AutoModel, AutoTokenizer
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large")

/Users/sina/Documents/GitHub_Local/uncovering_llm_generated_code/.venv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
# with open('output/rewrite/gemini_flash/rewrite_csn_gpt-4o_gemini-1.5-flash_50.json', 'r') as file:
#     data = json.load(file)

In [19]:
# Directory containing the JSON files
directory_path = 'output/rewrite/gpt_mini_12t'

# Initialize an empty list to store the combined data
data = []

# Iterate over all JSON files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.json'):  # Process only JSON files
        file_path = os.path.join(directory_path, filename)
        # print(file_path)
        with open(file_path, 'r') as file:
            file_data = json.load(file)  # Load JSON content
            data.extend(file_data)  # Add content to 

In [23]:
data1 = []
# Iterate over all JSON files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.json'):  # Process only JSON files
        file_path = os.path.join(directory_path, filename)
        # print(file_path)
        with open(file_path, 'r') as file:
            file_data = json.load(file)  # Load JSON content
            data1.append({'data': file_data, 
                         'dataset': file_path.split('/')[-1].split('.')[0]})  # Add content to 


In [4]:
detector_llm = directory_path.split('/')[-1]

In [5]:
import pandas as pd

In [6]:
detector_llm

'gpt_mini_12t'

In [20]:
data

[{'id': 2,
  'prompt': 'def selectOptimalChunk(self, peer):\n        """\n        select an optimal chunk to send to a peer.\n\n        @return: int(chunkNumber), str(chunkData) if there is data to be sent,\n        otherwise None, None\n        """',
  'code': 'def selectOptimalChunk(self, peer):\n    """\n    select an optimal chunk to send to a peer.\n\n    @return: int(chunkNumber), str(chunkData) if there is data to be sent,\n    otherwise None, None\n    """\n    available_chunks = self.getAvailableChunks(peer)\n    if not available_chunks:\n        return None, None\n    \n    optimal_chunk = min(available_chunks, key=lambda chunk: chunk[\'priority\'])\n    chunk_number = optimal_chunk[\'number\']\n    chunk_data = self.getChunkData(chunk_number)\n    \n    return chunk_number, chunk_data',
  'writer': 'machine',
  'rewrite': ["def select_optimal_chunk(self, peer):\n    available_chunks = self.get_available_chunks(peer)\n    if not available_chunks:\n        return None, None\n 

In [21]:
def string_similarity(first, second):
	output = subprocess.check_output(["node", "string-similarity-wrapper.js", first, second], universal_newlines=True)
	
	score = float(output.split(" ")[-1])
	return score

In [22]:
def classify_package(item, num_rewrites, strategy):
	texts = [item['code']]
	texts.extend(item['rewrite'])

	sim_scores = []

	if strategy == 'simcse':
		inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
		with torch.no_grad():
			embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

		for i in range(num_rewrites):
			sim = 1 - cosine(embeddings[0], embeddings[i+1])
			sim_scores.append(sim)

	elif strategy == 'string':
		for i in range(num_rewrites):
			sim = string_similarity(texts[0], texts[i+1])
			sim_scores.append(sim)

	elif strategy == 'tsed':
		for i in range(num_rewrites):
			sim = TSED.Calculate("python", texts[0], texts[i+1], 1.0, 0.8, 1.0)
			sim_scores.append(sim)
	else:
		print('ERROR. Invalid Strategy: "' + strategy + '"')
		return []

	return sim_scores

In [10]:
# print(f"Machine sample scores (simcse): {classify_package(data[0], 3, 'simcse')}") 
# print(f"Human sample scores(simcse): {classify_package(data[1], 3, 'simcse')}")


In [11]:
# print(f"Machine sample scores (tsed): {classify_package(data[0], 3, 'tsed')}")
# print(f"Machine sample scores (tsed): {classify_package(data[2], 3, 'tsed')}") 
# print(f"Machine sample scores (tsed): {classify_package(data[5], 3, 'tsed')}") 
# print(f"Machine sample scores (tsed): {classify_package(data[6], 3, 'tsed')}")
# print(f"Machine sample scores (tsed): {classify_package(data[9], 3, 'tsed')}") 
 

# print(f"Human sample scores(tsed): {classify_package(data[1], 3, 'tsed')}")
# print(f"Human sample scores(tsed): {classify_package(data[3], 3, 'tsed')}")
# print(f"Human sample scores(tsed): {classify_package(data[4], 3, 'tsed')}")
# print(f"Human sample scores(tsed): {classify_package(data[7], 3, 'tsed')}")
# print(f"Human sample scores(tsed): {classify_package(data[8], 3, 'tsed')}")

## Scoring the entire samples

In [23]:
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm for the progress bar

In [24]:
def scoring_process(strategy='simcse'):
    # Initialize lists to collect labels and scores
    labels = []
    scores = []

    # Choose the strategy for similarity computation
    strategy = strategy  # You can also use 'string' or 'tsed'

    # Iterate over the dataset with a progress bar
    for item in tqdm(data, desc="Processing items"):
        # Get the label (1 for machine, 0 for human)
        label = 1 if item['writer'] == 'machine' else 0
        labels.append(label)
        
        # Compute similarity scores using classify_package
        num_rewrites = len(item['rewrite'])
        sim_scores = classify_package(item, num_rewrites, strategy)
        
        # Average the similarity scores as per the formula
        avg_score = np.mean(sim_scores)
        scores.append(avg_score)

    # Convert lists to numpy arrays for compatibility
    labels = np.array(labels)
    scores = np.array(scores)

    # Compute ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(labels, scores)
    roc_auc = auc(fpr, tpr)

    # Print AUC value
    print(f'AUC for strategy "{strategy}": {roc_auc:.4f}')

    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Diagonal line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic - {strategy}')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score

def scoring_process(strategy='simcse', data=[]):
    # Initialize lists to collect labels and scores
    labels = []
    scores = []

    # Choose the strategy for similarity computation
    strategy = strategy  # You can also use 'string' or 'tsed'

    # Iterate over the dataset with a progress bar
    for item in tqdm(data, desc="Processing items"):
        # Get the label (1 for machine, 0 for human)
        label = 1 if item['writer'] == 'machine' else 0
        labels.append(label)
        
        # Compute similarity scores using classify_package
        num_rewrites = len(item['rewrite'])
        sim_scores = classify_package(item, num_rewrites, strategy)
        
        # Average the similarity scores as per the formula
        avg_score = np.mean(sim_scores)
        scores.append(avg_score)
        
        if item['id'] > 10:
            break

    # Convert lists to numpy arrays for compatibility
    labels = np.array(labels)
    scores = np.array(scores)

    # Compute ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(labels, scores)
    roc_auc = auc(fpr, tpr)
    
    # Find the optimal threshold (Youden's J statistic)
    J = tpr - fpr
    ix = np.argmax(J)
    optimal_threshold = thresholds[ix]
    print(f"Optimal Threshold: {optimal_threshold}")

    # Binarize the scores using the optimal threshold
    predicted_labels = (scores >= optimal_threshold).astype(int)
    
    # Compute accuracy, precision, and recall
    accuracy = accuracy_score(labels, predicted_labels)
    precision = precision_score(labels, predicted_labels)
    recall = recall_score(labels, predicted_labels)
    
    return roc_auc, accuracy, precision, recall


In [28]:
scoring_process(strategy='simcse', data=data)

Processing items:   2%|▏         | 11/600 [00:33<29:53,  3.05s/it] 

Optimal Threshold: 0.8504109978675842


(np.float64(0.3125), 0.6666666666666666, np.float64(0.7), np.float64(0.875))

In [13]:
# scoring_process(strategy='string')

In [14]:
# scoring_process(strategy='tsed')